In [1]:
import pandas as pd
import numpy as np
from textblob import TextBlob
from textblob.sentiments import PatternAnalyzer
import regex as re

In [2]:
BH_tweets_during_pandemic = pd.read_csv('dane/BH_tweets_during_pandemic.csv', lineterminator='\n')

In [3]:
BH_tweets_during_pandemic.describe()

,id,conversation_id,timezone,user_id,replies_count,retweets_count,likes_count,video,near,geo,source,user_rt_id,user_rt,retweet_id,retweet_date,translate,trans_src,trans_dest
count,6.617700e+04,6.617700e+04,66177.0,6.617700e+04,66177.000000,66177.000000,66177.000000,66177.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
mean,1.253162e+18,1.252844e+18,200.0,3.583153e+17,0.423350,1.264699,4.821494,0.345830,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,3.495924e+16,3.681137e+16,0.0,4.911625e+17,9.874074,25.475570,85.023278,0.475642,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,1.178838e+18,8.861777e+16,200.0,1.200000e+01,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,1.230565e+18,1.230467e+18,200.0,1.560364e+08,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,1.256720e+18,1.256711e+18,200.0,1.479996e+09,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,1.281920e+18,1.281828e+18,200.0,8.801123e+17,0.000000,0.000000,1.000000,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
max,1.311455e+18,1.311455e+18,200.0,1.310660e+18,2348.000000,5238.000000,15963.000000,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
BH_tweets_during_pandemic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66177 entries, 0 to 66176
Data columns (total 36 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id               66177 non-null  int64  
 1   conversation_id  66177 non-null  int64  
 2   created_at       66177 non-null  object 
 3   date             66177 non-null  object 
 4   time             66177 non-null  object 
 5   timezone         66177 non-null  int64  
 6   user_id          66177 non-null  int64  
 7   username         66177 non-null  object 
 8   name             66175 non-null  object 
 9   place            654 non-null    object 
 10  tweet            66177 non-null  object 
 11  language         66177 non-null  object 
 12  mentions         66177 non-null  object 
 13  urls             66177 non-null  object 
 14  photos           66177 non-null  object 
 15  replies_count    66177 non-null  int64  
 16  retweets_count   66177 non-null  int64  
 17  likes_count 

In [5]:
BH_tweets_during_pandemic.head()

,id,conversation_id,created_at,date,time,timezone,user_id,username,name,place,...,geo,source,user_rt_id,user_rt,retweet_id,reply_to,retweet_date,translate,trans_src,trans_dest
0,1311454882008891393,1311454882008891393,2020-10-01 01:56:24 CEST,2020-10-01,01:56:24,200,1262819762662301696,sells_san,SueSellsSanDiego,NaN,...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
1,1311451801011531776,1311451801011531776,2020-10-01 01:44:09 CEST,2020-10-01,01:44:09,200,485907955,wmwpres,Dave (WildMan) Weber,NaN,...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
2,1311451427559157762,1311451427559157762,2020-10-01 01:42:40 CEST,2020-10-01,01:42:40,200,332364947,simonsalhanyre,BHHS Simon & Salhany,NaN,...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
3,1311450245226692609,1311450245226692609,2020-10-01 01:37:58 CEST,2020-10-01,01:37:58,200,221858096,yourchgorealtor,Carla Walker,NaN,...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
4,1311448341998383105,1311448341998383105,2020-10-01 01:30:24 CEST,2020-10-01,01:30:24,200,289964571,bhhspenfedks,BHHS PenFed Realty Kansas,NaN,...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN


In [6]:
#Add date format and sorted by dates
BH_tweets_during_pandemic['date'] = pd.to_datetime(BH_tweets_during_pandemic['date'], format = '%Y-%m-%d')
BH_tweets_during_pandemic = BH_tweets_during_pandemic.sort_values(by='date')
BH_tweets_during_pandemic = BH_tweets_during_pandemic.reset_index(drop=True)
BH_tweets_during_pandemic

,id,conversation_id,created_at,date,time,timezone,user_id,username,name,place,...,geo,source,user_rt_id,user_rt,retweet_id,reply_to,retweet_date,translate,trans_src,trans_dest
0,1178838255221067779,1178838255221067779,2019-10-01 03:05:16 CEST,2019-10-01,03:05:16,200,1666180093,themillerteam_,Michelle Miller,NaN,...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
1,1179110500774236160,1179110500774236160,2019-10-01 21:07:04 CEST,2019-10-01,21:07:04,200,728971160,tagnifi,TagniFi,NaN,...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
2,1179110376023248901,1179110376023248901,2019-10-01 21:06:35 CEST,2019-10-01,21:06:35,200,811315902868295680,aliciacabezuela,Alicia Cabezuela,NaN,...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
3,1179108843818754049,1179108843818754049,2019-10-01 21:00:29 CEST,2019-10-01,21:00:29,200,24740419,gracepatalano,Grace Patalano,NaN,...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
4,1179107830789529600,1179107830789529600,2019-10-01 20:56:28 CEST,2019-10-01,20:56:28,200,800751574834233344,fmrealtorsteph,Stephanie Sarabakhsh,NaN,...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66172,1311428662693163008,1311428662693163008,2020-10-01 00:12:13 CEST,2020-10-01,00:12:13,200,351881121,christophercud1,Chris,NaN,...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
66173,1311426852825763840,1311426852825763840,2020-10-01 00:05:01 CEST,2020-10-01,00:05:01,200,351881121,christophercud1,Chris,NaN,...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
66174,1311425775929831427,1311425775929831427,2020-10-01 00:00:44 CEST,2020-10-01,00:00:44,200,800033690474659840,financedpremium,Financed Premiums,NaN,...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
66175,1311438444799893504,1311438444799893504,2020-10-01 00:51:05 CEST,2020-10-01,00:51:05,200,723604032141561856,dalesells30a,Dale Stackable,NaN,...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN


In [7]:
#Remove tweets writeen in other language than english
BH_tweets_during_pandemic_clean = BH_tweets_during_pandemic[BH_tweets_during_pandemic.language == 'en']
BH_tweets_during_pandemic_clean = BH_tweets_during_pandemic_clean.reset_index(drop=True)
BH_tweets_during_pandemic_clean.shape

(59650, 36)

In [8]:
#Remove columns without any data
BH_tweets_during_pandemic_clean = BH_tweets_during_pandemic_clean.dropna(thresh = 1, axis = 1)

In [9]:
#6527 records and 10 columns were removed
BH_tweets_during_pandemic_clean.describe()

,id,conversation_id,timezone,user_id,replies_count,retweets_count,likes_count,video
count,5.965000e+04,5.965000e+04,59650.0,5.965000e+04,59650.000000,59650.000000,59650.000000,59650.000000
mean,1.252787e+18,1.252444e+18,200.0,3.552008e+17,0.412372,1.223470,4.544912,0.361459
std,3.515947e+16,3.718885e+16,0.0,4.882400e+17,10.379881,26.456237,88.666740,0.480427
min,1.178838e+18,8.861777e+16,200.0,1.200000e+01,0.000000,0.000000,0.000000,0.000000
25%,1.229553e+18,1.229436e+18,200.0,1.572161e+08,0.000000,0.000000,0.000000,0.000000
50%,1.256690e+18,1.256687e+18,200.0,1.466309e+09,0.000000,0.000000,0.000000,0.000000
75%,1.281710e+18,1.281660e+18,200.0,8.576936e+17,0.000000,0.000000,0.000000,1.000000
max,1.311455e+18,1.311455e+18,200.0,1.310537e+18,2348.000000,5238.000000,15963.000000,1.000000


In [10]:
BH_tweets_during_pandemic_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59650 entries, 0 to 59649
Data columns (total 26 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   id               59650 non-null  int64         
 1   conversation_id  59650 non-null  int64         
 2   created_at       59650 non-null  object        
 3   date             59650 non-null  datetime64[ns]
 4   time             59650 non-null  object        
 5   timezone         59650 non-null  int64         
 6   user_id          59650 non-null  int64         
 7   username         59650 non-null  object        
 8   name             59648 non-null  object        
 9   place            640 non-null    object        
 10  tweet            59650 non-null  object        
 11  language         59650 non-null  object        
 12  mentions         59650 non-null  object        
 13  urls             59650 non-null  object        
 14  photos           59650 non-null  objec

In [11]:
BH_tweets_during_pandemic_clean.head()

,id,conversation_id,created_at,date,time,timezone,user_id,username,name,place,...,retweets_count,likes_count,hashtags,cashtags,link,retweet,quote_url,video,thumbnail,reply_to
0,1178838255221067779,1178838255221067779,2019-10-01 03:05:16 CEST,2019-10-01,03:05:16,200,1666180093,themillerteam_,Michelle Miller,NaN,...,0,0,[],[],https://twitter.com/TheMillerTeam_/status/1178...,False,NaN,1,https://pbs.twimg.com/media/EFwTJWEWoAEGRXA.jpg,[]
1,1179110500774236160,1179110500774236160,2019-10-01 21:07:04 CEST,2019-10-01,21:07:04,200,728971160,tagnifi,TagniFi,NaN,...,0,0,[],[],https://twitter.com/tagnifi/status/11791105007...,False,NaN,1,https://pbs.twimg.com/media/EF0KojVUEAAgGsL.jpg,[]
2,1179110376023248901,1179110376023248901,2019-10-01 21:06:35 CEST,2019-10-01,21:06:35,200,811315902868295680,aliciacabezuela,Alicia Cabezuela,NaN,...,0,0,[],[],https://twitter.com/AliciaCabezuela/status/117...,False,NaN,1,https://pbs.twimg.com/ext_tw_video_thumb/11791...,[]
3,1179108843818754049,1179108843818754049,2019-10-01 21:00:29 CEST,2019-10-01,21:00:29,200,24740419,gracepatalano,Grace Patalano,NaN,...,0,0,[],[],https://twitter.com/gracepatalano/status/11791...,False,NaN,1,https://pbs.twimg.com/media/EF0JPrdW4AEIftW.jpg,[]
4,1179107830789529600,1179107830789529600,2019-10-01 20:56:28 CEST,2019-10-01,20:56:28,200,800751574834233344,fmrealtorsteph,Stephanie Sarabakhsh,NaN,...,0,0,"['travel', 'vacation']",[],https://twitter.com/FMRealtorSteph/status/1179...,False,NaN,1,https://pbs.twimg.com/media/EF0IUrbWsAU8qm2.jpg,[]


In [12]:
#Check values of column place
BH_tweets_during_pandemic_clean['place'].value_counts()

{'type': 'Point', 'coordinates': [44.6981658, -93.2177749]}      120
{'type': 'Point', 'coordinates': [41.25877517, -96.19294167]}     44
{'type': 'Point', 'coordinates': [30.32487, -81.39882]}           23
{'type': 'Point', 'coordinates': [46.922594, -97.552256]}         12
{'type': 'Point', 'coordinates': [41.883222, -87.632496]}         10
                                                                ... 
{'type': 'Point', 'coordinates': [46.593, -120.5378]}              1
{'type': 'Point', 'coordinates': [41.782313, -86.7802619]}         1
{'type': 'Point', 'coordinates': [42.6887208, -84.2761674]}        1
{'type': 'Point', 'coordinates': [42.737325, -84.4392809]}         1
{'type': 'Point', 'coordinates': [42.736889, -83.584152]}          1
Name: place, Length: 280, dtype: int64

In [13]:
#Check values of column quote_url
BH_tweets_during_pandemic_clean['quote_url'].value_counts()

0                                                               76
https://twitter.com/trengriffin/status/1256687728743202819       9
https://twitter.com/CNBCnow/status/1246193229546033152           6
https://twitter.com/business/status/1281698952384065538          5
https://twitter.com/CNBC/status/1279852881403936769              5
                                                                ..
https://twitter.com/acquirersx/status/1258698207401594880        1
https://twitter.com/NoahShachtman/status/1280220757801476098     1
https://twitter.com/MylesUdland/status/1256719159540568064       1
https://twitter.com/KSHB41/status/1270524555220652032            1
https://twitter.com/Reuters/status/1231320254665506816           1
Name: quote_url, Length: 1266, dtype: int64

In [14]:
#Counted quantity of links for each tweet
BH_tweets_during_pandemic_clean['quote_url'] = BH_tweets_during_pandemic_clean['quote_url'].str.count('https://')

#Replayce NaN by 0
BH_tweets_during_pandemic_clean.fillna({'quote_url':0}, inplace=True)

#Changed data type from float64 to int64
BH_tweets_during_pandemic_clean['quote_url'] = BH_tweets_during_pandemic_clean['quote_url'].astype(np.int64)

In [15]:
#Replayce NaN to [] in thumbnail column
BH_tweets_during_pandemic_clean.fillna({'thumbnail':'[]'}, inplace=True)

In [16]:
#There is no NaN except column place which will be removed due to low importance
BH_tweets_during_pandemic_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59650 entries, 0 to 59649
Data columns (total 26 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   id               59650 non-null  int64         
 1   conversation_id  59650 non-null  int64         
 2   created_at       59650 non-null  object        
 3   date             59650 non-null  datetime64[ns]
 4   time             59650 non-null  object        
 5   timezone         59650 non-null  int64         
 6   user_id          59650 non-null  int64         
 7   username         59650 non-null  object        
 8   name             59648 non-null  object        
 9   place            640 non-null    object        
 10  tweet            59650 non-null  object        
 11  language         59650 non-null  object        
 12  mentions         59650 non-null  object        
 13  urls             59650 non-null  object        
 14  photos           59650 non-null  objec

In [17]:
#Check duplicates
BH_tweets_during_pandemic_clean[BH_tweets_during_pandemic_clean.duplicated(subset = 'id')]

,id,conversation_id,created_at,date,time,timezone,user_id,username,name,place,...,retweets_count,likes_count,hashtags,cashtags,link,retweet,quote_url,video,thumbnail,reply_to


In [18]:
#Column retweet consist of only False values
BH_tweets_during_pandemic_clean['retweet'].value_counts()

False    59650
Name: retweet, dtype: int64

In [19]:
#Removed irrelevant columns
BH_tweets_during_pandemic_clean = BH_tweets_during_pandemic_clean.drop(['conversation_id', 
                                                                        'created_at', 'time', 'timezone', 
                                                                        'language', 'retweet', 'link','place'], 
                                                                       axis = 1)

In [20]:
#Counted number of elements stored as lists in columns
columns_lists = ['mentions', 'urls', 'photos', 'hashtags', 'cashtags', 'thumbnail']
for column in columns_lists:
    empty_list = BH_tweets_during_pandemic_clean[column] == '[]'
    BH_tweets_during_pandemic_clean[column] = BH_tweets_during_pandemic_clean[column].str[1:-1].str.split(',')
    BH_tweets_during_pandemic_clean[column] = BH_tweets_during_pandemic_clean[column].str.len()
    BH_tweets_during_pandemic_clean.loc[empty_list, column] = 0

In [21]:
#Counted number of elements in dictionaries stored in reply_to column
BH_tweets_during_pandemic_clean['reply_to'] = BH_tweets_during_pandemic_clean['reply_to'].str.count('{')

#Changed data type from float64 to int64
BH_tweets_during_pandemic_clean['reply_to'] = BH_tweets_during_pandemic_clean['reply_to'].astype(np.int64)

In [22]:
#There are 2 name missing
BH_tweets_during_pandemic_clean[BH_tweets_during_pandemic_clean['name'].isnull()]

,id,date,user_id,username,name,tweet,mentions,urls,photos,replies_count,retweets_count,likes_count,hashtags,cashtags,quote_url,video,thumbnail,reply_to
31031,1256981981075632128,2020-05-03,2259884294,stocks_bot,NaN,"How Warren Buffett's 'mistake,' Berkshire Hath...",0,1,0,0,0,0,0,0,0,0,0,0
37491,1266058798394793985,2020-05-28,2259884294,stocks_bot,NaN,Why hedge-fund manager Bill Ackman says he dum...,0,1,0,0,0,0,0,0,0,0,0,0


In [23]:
#Replayce missing names by stocks_bot
BH_tweets_during_pandemic_clean.fillna({'name':'stocks_bot'}, inplace=True)

In [24]:
#Removed hastags, mentions, url and special signs from tweets
for i in range(BH_tweets_during_pandemic_clean.shape[0]):
    BH_tweets_during_pandemic_clean.loc[i, 'tweet'] = ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ",
                            BH_tweets_during_pandemic_clean.loc[i, 'tweet']).split())

In [25]:
#Last day 2020-10-01 has only 25 tweets, it will be removed
BH_tweets_during_pandemic_clean['date'].value_counts()

2020-05-03    2241
2020-05-02    1684
2020-05-04     898
2020-08-31     862
2020-03-14     663
              ... 
2020-01-04      66
2019-11-30      66
2019-10-26      65
2019-12-01      59
2020-10-01      22
Name: date, Length: 367, dtype: int64

In [26]:
BH_tweets_during_pandemic_clean = BH_tweets_during_pandemic_clean[BH_tweets_during_pandemic_clean['date'] != '2020-10-01']
BH_tweets_during_pandemic_clean['date'].value_counts()

2020-05-03    2241
2020-05-02    1684
2020-05-04     898
2020-08-31     862
2020-03-14     663
              ... 
2020-01-04      66
2019-11-30      66
2019-10-13      66
2019-10-26      65
2019-12-01      59
Name: date, Length: 366, dtype: int64

In [27]:
#Checked if there are tweets that contains only white spaces, numbers or special signs
BH_tweets_during_pandemic_clean[~BH_tweets_during_pandemic_clean["tweet"].str.contains('[A-Za-z]')]

,id,date,user_id,username,name,tweet,mentions,urls,photos,replies_count,retweets_count,likes_count,hashtags,cashtags,quote_url,video,thumbnail,reply_to


In [28]:
BH_tweets_during_pandemic_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 59628 entries, 0 to 59627
Data columns (total 18 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   id              59628 non-null  int64         
 1   date            59628 non-null  datetime64[ns]
 2   user_id         59628 non-null  int64         
 3   username        59628 non-null  object        
 4   name            59628 non-null  object        
 5   tweet           59628 non-null  object        
 6   mentions        59628 non-null  int64         
 7   urls            59628 non-null  int64         
 8   photos          59628 non-null  int64         
 9   replies_count   59628 non-null  int64         
 10  retweets_count  59628 non-null  int64         
 11  likes_count     59628 non-null  int64         
 12  hashtags        59628 non-null  int64         
 13  cashtags        59628 non-null  int64         
 14  quote_url       59628 non-null  int64         
 15  vi

In [29]:
#Save cleaned data in csv format
BH_tweets_during_pandemic_clean.to_csv('dane/BH_tweets_during_pandemic_clean.csv', index=False)

In [30]:
#Create dataframe for textanalysis for each day
BH_tweets_during_pandemic_textanalysis = pd.date_range(BH_tweets_during_pandemic_clean['date'].min(),
                                                       BH_tweets_during_pandemic_clean['date'].max(),freq='d')
BH_tweets_during_pandemic_textanalysis = pd.DataFrame(BH_tweets_during_pandemic_textanalysis, columns = ['date'])
column_names = ['Short negative factual tweets', 'Short negative personal tweets', 'Short positive factual tweets',
                'Short positive personal tweets', 'Short neutral factual tweets', 'Short neutral personal tweets',
                'Long negative factual tweets', 'Long negative personal tweets', 'Long positive factual tweets',
                'Long positive personal tweets', 'Long neutral factual tweets', 'Long neutral personal tweets',
                'Total amount of tweets']
for column in column_names:
    BH_tweets_during_pandemic_textanalysis[column] = 0
BH_tweets_during_pandemic_textanalysis

,date,Short negative factual tweets,Short negative personal tweets,Short positive factual tweets,Short positive personal tweets,Short neutral factual tweets,Short neutral personal tweets,Long negative factual tweets,Long negative personal tweets,Long positive factual tweets,Long positive personal tweets,Long neutral factual tweets,Long neutral personal tweets,Total amount of tweets
0,2019-10-01,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2019-10-02,0,0,0,0,0,0,0,0,0,0,0,0,0
2,2019-10-03,0,0,0,0,0,0,0,0,0,0,0,0,0
3,2019-10-04,0,0,0,0,0,0,0,0,0,0,0,0,0
4,2019-10-05,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
361,2020-09-26,0,0,0,0,0,0,0,0,0,0,0,0,0
362,2020-09-27,0,0,0,0,0,0,0,0,0,0,0,0,0
363,2020-09-28,0,0,0,0,0,0,0,0,0,0,0,0,0
364,2020-09-29,0,0,0,0,0,0,0,0,0,0,0,0,0


In [31]:
#Analysis of opinion and number of words using the textblob package
for i in range(BH_tweets_during_pandemic_clean.shape[0]):
    blob = TextBlob(BH_tweets_during_pandemic_clean.loc[i, 'tweet'], analyzer=PatternAnalyzer())
    if len(blob.words) < 30:
        if blob.sentiment[0] < 0:
            if blob.sentiment[1] < 0.5:
                BH_tweets_during_pandemic_textanalysis.loc[BH_tweets_during_pandemic_textanalysis[BH_tweets_during_pandemic_textanalysis['date'] == BH_tweets_during_pandemic_clean.loc[i, 'date']].index.values[0], 
                                                               'Short negative factual tweets'] += 1
            else:
                BH_tweets_during_pandemic_textanalysis.loc[BH_tweets_during_pandemic_textanalysis[BH_tweets_during_pandemic_textanalysis['date'] == BH_tweets_during_pandemic_clean.loc[i, 'date']].index.values[0], 
                                                               'Short negative personal tweets'] += 1
        elif blob.sentiment[0] > 0:
            if blob.sentiment[1] < 0.5:
                BH_tweets_during_pandemic_textanalysis.loc[BH_tweets_during_pandemic_textanalysis[BH_tweets_during_pandemic_textanalysis['date'] == BH_tweets_during_pandemic_clean.loc[i, 'date']].index.values[0], 
                                                               'Short positive factual tweets'] += 1
            else:
                BH_tweets_during_pandemic_textanalysis.loc[BH_tweets_during_pandemic_textanalysis[BH_tweets_during_pandemic_textanalysis['date'] == BH_tweets_during_pandemic_clean.loc[i, 'date']].index.values[0], 
                                                               'Short positive personal tweets'] += 1
        else:
            if blob.sentiment[1] < 0.5:
                BH_tweets_during_pandemic_textanalysis.loc[BH_tweets_during_pandemic_textanalysis[BH_tweets_during_pandemic_textanalysis['date'] == BH_tweets_during_pandemic_clean.loc[i, 'date']].index.values[0], 
                                                               'Short neutral factual tweets'] += 1
            else:
                BH_tweets_during_pandemic_textanalysis.loc[BH_tweets_during_pandemic_textanalysis[BH_tweets_during_pandemic_textanalysis['date'] == BH_tweets_during_pandemic_clean.loc[i, 'date']].index.values[0], 
                                                               'Short neutral personal tweets'] += 1
    else:
        if blob.sentiment[0] < 0:
            if blob.sentiment[1] < 0.5:
                BH_tweets_during_pandemic_textanalysis.loc[BH_tweets_during_pandemic_textanalysis[BH_tweets_during_pandemic_textanalysis['date'] == BH_tweets_during_pandemic_clean.loc[i, 'date']].index.values[0], 
                                                               'Long negative factual tweets'] += 1
            else:
                BH_tweets_during_pandemic_textanalysis.loc[BH_tweets_during_pandemic_textanalysis[BH_tweets_during_pandemic_textanalysis['date'] == BH_tweets_during_pandemic_clean.loc[i, 'date']].index.values[0], 
                                                               'Long negative personal tweets'] += 1
        elif blob.sentiment[0] > 0:
            if blob.sentiment[1] < 0.5:
                BH_tweets_during_pandemic_textanalysis.loc[BH_tweets_during_pandemic_textanalysis[BH_tweets_during_pandemic_textanalysis['date'] == BH_tweets_during_pandemic_clean.loc[i, 'date']].index.values[0], 
                                                               'Long positive factual tweets'] += 1
            else:
                BH_tweets_during_pandemic_textanalysis.loc[BH_tweets_during_pandemic_textanalysis[BH_tweets_during_pandemic_textanalysis['date'] == BH_tweets_during_pandemic_clean.loc[i, 'date']].index.values[0], 
                                                               'Long positive personal tweets'] += 1
        else:
            if blob.sentiment[1] < 0.5:
                BH_tweets_during_pandemic_textanalysis.loc[BH_tweets_during_pandemic_textanalysis[BH_tweets_during_pandemic_textanalysis['date'] == BH_tweets_during_pandemic_clean.loc[i, 'date']].index.values[0], 
                                                               'Long neutral factual tweets'] += 1
            else:
                BH_tweets_during_pandemic_textanalysis.loc[BH_tweets_during_pandemic_textanalysis[BH_tweets_during_pandemic_textanalysis['date'] == BH_tweets_during_pandemic_clean.loc[i, 'date']].index.values[0], 
                                                               'Long neutral personal tweets'] += 1
    BH_tweets_during_pandemic_textanalysis.loc[BH_tweets_during_pandemic_textanalysis[BH_tweets_during_pandemic_textanalysis['date'] == BH_tweets_during_pandemic_clean.loc[i, 'date']].index.values[0], 
                                                               'Total amount of tweets'] += 1

In [32]:
BH_tweets_during_pandemic_textanalysis

,date,Short negative factual tweets,Short negative personal tweets,Short positive factual tweets,Short positive personal tweets,Short neutral factual tweets,Short neutral personal tweets,Long negative factual tweets,Long negative personal tweets,Long positive factual tweets,Long positive personal tweets,Long neutral factual tweets,Long neutral personal tweets,Total amount of tweets
0,2019-10-01,4,1,8,13,16,0,3,2,5,12,8,0,72
1,2019-10-02,4,0,13,12,23,1,0,1,13,8,3,1,79
2,2019-10-03,4,1,11,19,44,0,5,2,18,12,8,1,125
3,2019-10-04,2,1,10,15,40,5,1,0,11,9,4,1,99
4,2019-10-05,0,1,8,17,15,0,1,4,4,10,8,3,71
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
361,2020-09-26,7,1,14,16,34,1,1,1,16,6,3,0,100
362,2020-09-27,2,2,8,11,24,1,4,1,12,8,3,0,76
363,2020-09-28,8,2,5,19,60,0,1,3,18,14,7,0,137
364,2020-09-29,20,8,19,27,47,0,3,4,30,19,6,0,183


In [33]:
BH_tweets_during_pandemic_textanalysis.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 366 entries, 0 to 365
Data columns (total 14 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   date                            366 non-null    datetime64[ns]
 1   Short negative factual tweets   366 non-null    int64         
 2   Short negative personal tweets  366 non-null    int64         
 3   Short positive factual tweets   366 non-null    int64         
 4   Short positive personal tweets  366 non-null    int64         
 5   Short neutral factual tweets    366 non-null    int64         
 6   Short neutral personal tweets   366 non-null    int64         
 7   Long negative factual tweets    366 non-null    int64         
 8   Long negative personal tweets   366 non-null    int64         
 9   Long positive factual tweets    366 non-null    int64         
 10  Long positive personal tweets   366 non-null    int64         
 11  Long n

In [34]:
#Save text analysis data in csv format
BH_tweets_during_pandemic_textanalysis.to_csv('dane/BH_tweets_during_pandemic_textanalysis.csv', index=False)